# Sales Data

In [13]:
# Importing the required modules
from flask import Flask, render_template, jsonify
from flask_pymongo import PyMongo
import pandas as pd
import json

In [37]:
# Extracting Sales csv file into dataframe
test_file = "Data/IEA-EV-dataEV salesCarsHistorical.csv"
test_df = pd.read_csv(test_file)
    
# Clean Sales dataset by dropping PHEV Powertrain
test_df=test_df.loc[(test_df['powertrain'] == 'BEV')].reset_index(drop=True)

In [38]:
test_df

,region,category,parameter,mode,powertrain,year,unit,value
0,Australia,Historical,EV sales,Cars,BEV,2011,sales,49
1,Australia,Historical,EV sales,Cars,BEV,2012,sales,170
2,Australia,Historical,EV sales,Cars,BEV,2013,sales,190
3,Australia,Historical,EV sales,Cars,BEV,2014,sales,370
4,Australia,Historical,EV sales,Cars,BEV,2015,sales,760
...,...,...,...,...,...,...,...,...
340,World,Historical,EV sales,Cars,BEV,2017,sales,760000
341,World,Historical,EV sales,Cars,BEV,2018,sales,1400000
342,World,Historical,EV sales,Cars,BEV,2019,sales,1500000
343,World,Historical,EV sales,Cars,BEV,2020,sales,2000000


In [30]:
def sales():
    
    # Extracting Sales csv file into dataframe
    sales_file = "Data/IEA-EV-dataEV salesCarsHistorical.csv"
    sales_df = pd.read_csv(sales_file)
    
    # Clean Sales dataset by dropping PHEV Powertrain
    sales_df=sales_df.loc[(sales_df['powertrain'] == 'BEV')].reset_index(drop=True)

    # drop unused columns
    sales_df.drop(sales_df.columns[[1,2,3,4,6]], axis=1, inplace=True)

    # select only sales in the US
    sales_df = sales_df.loc[(sales_df['region'] == 'USA')].reset_index(drop=True)

    # change name to something meaningful
    sales_df.rename(columns={'value' : 'sold'}, inplace=True)

    # drop this column since it is no longer needed
    sales_df.drop(columns=['region'], inplace=True)

    return sales_df.to_dict('split')

In [31]:
# Create an instance of Flask
app = Flask(__name__)

In [32]:
# Setup connection to mongodb
conn = "mongodb://localhost:27017/ev_data"
mongo = PyMongo(app, uri = conn)

In [33]:
ev_sale = sales()

In [34]:
ev_sale

{'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
 'columns': ['year', 'sold'],
 'data': [[2010, 1200],
  [2011, 9800],
  [2012, 15000],
  [2013, 48000],
  [2014, 63000],
  [2015, 71000],
  [2016, 87000],
  [2017, 100000],
  [2018, 240000],
  [2019, 240000],
  [2020, 230000],
  [2021, 470000]]}

In [35]:
type(ev_sale)

dict

In [36]:
mongo.db.sales.update_one({}, {"$set": ev_sale}, upsert=True)

In [8]:
result = mongo.db.sales.find()

In [9]:
result_list = list(result)

In [10]:
result

In [11]:
if len(result_list) > 0:
        # Return the first result only and strip off the '_id'
        data = result_list[0] 
        id_to_discard = data.pop('_id', None)
else:
        # Construct an error message
        data = {'Error': 'No data found'} 

In [12]:

data

{'columns': ['index',
  'region',
  'category',
  'parameter',
  'mode',
  'powertrain',
  'year',
  'unit',
  'value'],
 'data': [[0,
   'Australia',
   'Historical',
   'EV sales',
   'Cars',
   'BEV',
   2011,
   'sales',
   49],
  [1,
   'Australia',
   'Historical',
   'EV sales',
   'Cars',
   'BEV',
   2012,
   'sales',
   170],
  [4,
   'Australia',
   'Historical',
   'EV sales',
   'Cars',
   'BEV',
   2013,
   'sales',
   190],
  [5,
   'Australia',
   'Historical',
   'EV sales',
   'Cars',
   'BEV',
   2014,
   'sales',
   370],
  [8,
   'Australia',
   'Historical',
   'EV sales',
   'Cars',
   'BEV',
   2015,
   'sales',
   760],
  [9,
   'Australia',
   'Historical',
   'EV sales',
   'Cars',
   'BEV',
   2016,
   'sales',
   670],
  [12,
   'Australia',
   'Historical',
   'EV sales',
   'Cars',
   'BEV',
   2017,
   'sales',
   1200],
  [13,
   'Australia',
   'Historical',
   'EV sales',
   'Cars',
   'BEV',
   2018,
   'sales',
   1800],
  [16,
   'Australia',
   'H